In [3]:
import os
import mne
import pycartool.io
%matplotlib qt

## Read EEG data

In [5]:
folder = fr'E:\switchdrive\Brainhack\KMR1\d3'
sef_file = 'KMR1_d3_Raw_DS.Avg_ref.sef'
sef_path = os.path.join(folder, sef_file)
raw = pycartool.io.read_sef(sef_path)
raw.plot(scalings='auto');

[sef.read_sef] INFO: Cannot read recording date from file: year 0 is out of range
Creating RawArray with float64 data, n_channels=16, n_times=15019520
    Range : 0 ... 15019519 =      0.000 ...  3754.880 secs
Ready.


Channels marked as bad:
none


## Read events

### Bads

In [41]:
import pandas as pd

bad_file = 'bad_epoch_v4.mrk'
bad_path = os.path.join(folder, bad_file)

df = pd.read_csv(bad_path, sep="\t", skiprows=1, names=['start', 'stop', 'label'])
df['start_time'] = df['start'] / raw.info['sfreq']
df['stop_time'] = df['stop'] / raw.info['sfreq']
df['duration'] = df['stop_time'] - df['start_time']
bads_annotations = mne.Annotations(df['start_time'], df['duration'], df['label'])

### Epileptic events

In [42]:
epileptic_events_file = 'Epileptic_events.mrk'
epileptic_events_path = os.path.join(folder, epileptic_events_file)

df = pd.read_csv(epileptic_events_path, sep="\t", skiprows=1, names=['start', 'stop', 'label'])
df['start_time'] = df['start'] / raw.info['sfreq']
df['stop_time'] = df['stop'] / raw.info['sfreq']
df['duration'] = df['stop_time'] - df['start_time']

events = list()
for r, row in df.iterrows():
    label = row['label']
    if label.lower() == 'hpd_start':
        start = row['start_time']
        for r_, row_ in df.iloc[r:].iterrows():
            label_ = row_['label']
            if label_.lower() == 'hpd_end':
                stop = row_['start_time']
                event = dict()
                event['label'] = 'hpd'
                event['start'] = start
                event['stop'] = stop
                event['duration'] = stop - start
                events.append(event)
                break

df_annot = pd.DataFrame(events)
epileptic_annotations = mne.Annotations(df_annot['start'], df_annot['duration'], df_annot['label'])


In [49]:
background_events_file = 'mrk_bck.mrk'
background_events_path = os.path.join(folder, background_events_file)

df = pd.read_csv(background_events_path, sep="\t", skiprows=1, names=['start', 'stop', 'label'])
df['start_time'] = df['start'] / raw.info['sfreq'] - 0.5
df['duration'] = 1 # 1sec of background around marker
background_annotations = mne.Annotations(df['start_time'], df['duration'], 'background')

In [50]:
annotations = epileptic_annotations + bads_annotations + background_annotations
raw.set_annotations(annotations) 

<RawArray | 16 x 15019520 (3754.9 s), ~1.79 GB, data loaded>

In [51]:
raw.plot(scalings='auto')

Channels marked as bad:
none
